<a href="https://colab.research.google.com/github/SANTOSHMAHER/Text-Analysis-and-Word-Predaction/blob/master/Prediction%2Bcalcification3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Embedding,Bidirectional,Dense,Dropout

import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow 
%matplotlib inline 
from keras import losses 
from keras.callbacks import ModelCheckpoint 

import re
import string 
import pandas as pd 
import numpy as np
import sys
import io
import os

Using TensorFlow backend.


In [0]:
df=pd.read_excel("2019_ted.xlsx")
df.head()

,Title,Description,Ref,TypeOfContract,Cpv (main),Cpv (additional)
0,Provision of consultancy services for Faciliti...,Where expert knowledge on complex facilities m...,FM/08/CS/MC/18,SERVICES,79993100,79993000
1,Bathymetry — High Resolution Seabed Mapping,EASME is launching an Invitation to Tender wit...,EASME/EMFF/2018/007,SERVICES,79000000,NaN
2,Assistance Mechanism for the Western Mediterra...,EASME is launching this call for tenders divid...,EASME/EMFF/2017/039,SERVICES,79000000,79000000
3,UCA 18/068 — Management of the crèche at the G...,See II.2.4).The General Secretariat of the Cou...,UCA 18/068,SERVICES,85312110,NaN
4,Technical Support for the Development and Upda...,EUROMOD is a tax-benefit microsimulation model...,JRC/SVQ/2018/B.2/0017/OC,SERVICES,72330000,"72330000,72300000"


In [0]:
df = df[["Title","Cpv (main)"]]
df.head()

,Title,Cpv (main)
0,Provision of consultancy services for Faciliti...,79993100
1,Bathymetry — High Resolution Seabed Mapping,79000000
2,Assistance Mechanism for the Western Mediterra...,79000000
3,UCA 18/068 — Management of the crèche at the G...,85312110
4,Technical Support for the Development and Upda...,72330000


In [0]:
df = df.rename(columns={"Cpv (main)":"CpvMain"})
df.head()

,Title,CpvMain
0,Provision of consultancy services for Faciliti...,79993100
1,Bathymetry — High Resolution Seabed Mapping,79000000
2,Assistance Mechanism for the Western Mediterra...,79000000
3,UCA 18/068 — Management of the crèche at the G...,85312110
4,Technical Support for the Development and Upda...,72330000


In [0]:
df.to_csv("ted.txt", index=False)

In [0]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r',encoding="utf-8")
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
doc = load_doc("ted.txt")
print(doc[:200])

Title,CpvMain
Provision of consultancy services for Facilities Management,79993100
Bathymetry — High Resolution Seabed Mapping,79000000
Assistance Mechanism for the Western Mediterranean and the Atlan


In [0]:
# turn a doc into clean tokens
def clean_doc(doc):
    doc = doc.replace('-',' ').replace('.0',' ').replace(',',' ')  # replace '--' with a space ' '
    tokens = doc.split()  # split into tokens by white space
    table = str.maketrans('', '', string.punctuation)  # remove punctuation from each token
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalnum()]  # remove remaining tokens that are not alphabetic
    tokens = [word.lower() for word in tokens]  # make lower case
    return tokens
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['title', 'cpvmain', 'provision', 'of', 'consultancy', 'services', 'for', 'facilities', 'management', '79993100', 'bathymetry', 'high', 'resolution', 'seabed', 'mapping', '79000000', 'assistance', 'mechanism', 'for', 'the', 'western', 'mediterranean', 'and', 'the', 'atlantic', 'action', 'plan', 'lot', '1', '2', '79000000', 'uca', '18068', 'management', 'of', 'the', 'crèche', 'at', 'the', 'general', 'secretariat', 'of', 'the', 'council', 'of', 'the', 'european', 'union', '85312110', 'technical', 'support', 'for', 'the', 'development', 'and', 'update', 'of', 'euromod', 'lots', '1', '2', '3', '4', '6', '7', '72330000', 'security', 'guard', 'and', 'receptionswitchboard', 'services', 'for', 'the', 'premises', 'of', 'the', 'eu', 'house', 'in', 'poland', '79710000', 'design', 'and', 'build', 'of', 'the', 'sustainable', 'living', 'complex', 'bldgs', 'substructure', 'inc', 'the', 'respective', 'surrounding', 'landscaping', 'infrastructure', 'part', 'finishes', 'mech', 'and', 'elect', 'wrks', 'i

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)
encoded = tokenizer.texts_to_sequences([tokens])[0]

In [0]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 8824


In [0]:
# encode 2 words -> 1 word
sequences = list()
for i in range(3, len(encoded)):
    sequence = encoded[i-3:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 47326


In [0]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length) 

Max Sequence Length: 4


In [0]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]  # encode the text as integer
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')  # pre-pad sequences to a fixed length
        yhat = model.predict_classes(encoded, verbose=0)  # predict probabilities for each word
        out_word = ''  # map predicted word index to word
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

In [0]:
from sklearn.externals import joblib
model1 = joblib.load('txt_pred_model2019.pkl')
tokenizer1 = joblib.load("txt_pred_tokenizer2019.pkl")

#load classification model to predict CpvMain category
model2 = joblib.load('ted_2019_clf.pkl')

W0628 18:10:24.040527 139695729440640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 18:10:24.097370 139695729440640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 18:10:24.126617 139695729440640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 18:10:24.443916 139695729440640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0628 18:10:24.445766 1396957294

In [0]:
print("Enter seeds for predict next words sequence")
string = input(str()) 

Enter seeds for predict next words sequence
Supply of Advanced Visual


In [0]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 10)             88240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               71168     
_________________________________________________________________
dense_1 (Dense)              (None, 8824)              1138296   
Total params: 1,297,704
Trainable params: 1,297,704
Non-trainable params: 0
_________________________________________________________________


In [0]:
# evaluate model
Predicted_title = generate_seq(model1, tokenizer1, max_length, string,12)
Predicted_title

'Supply of Advanced Visual docking guidance systems associated components integration software and ad hoc technical services'

In [0]:
print(Predicted_title)
print("CpvMain :", model2.predict([Predicted_title])[0])

Supply of Advanced Visual docking guidance systems associated components integration software and ad hoc technical services
CpvMain : 34960000
